In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain.tools import tool, ToolRuntime


@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]


@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [ ]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware


class EmailState(AgentState):
    email: str


agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [ ]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="Please read my email and send a response immediately. Send the reply now in the same thread."
            )
        ],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John.",
    },
    config=config,
)

In [5]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'No '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'the '
                                                                          'heads '
                                                                          'up. '
                                                                          'I’m '
                                                                          'happy '
                                                                          'to '
               

In [ ]:
print(response["__interrupt__"])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': 'Hi John,\n\nNo problem—thanks for the heads up. I’m happy to reschedule. Would 2:00 PM or 4:00 PM tomorrow work for you? If neither works, please let me know a couple of times that fit your schedule and I’ll adjust.\n\nBest regards,\nSeán'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': 'Hi John,\\n\\nNo problem—thanks for the heads up. I’m happy to reschedule. Would 2:00 PM or 4:00 PM tomorrow work for you? If neither works, please let me know a couple of times that fit your schedule and I’ll adjust.\\n\\nBest regards,\\nSeán'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='e20e67fb8a1c77860dc541bb9a1817a2')]


In [ ]:
# Access just the 'body' argument from the tool call
print(response["__interrupt__"][0].value["action_requests"][0]["args"]["body"])

Hi John,

No problem—thanks for the heads up. I’m happy to reschedule. Would 2:00 PM or 4:00 PM tomorrow work for you? If neither works, please let me know a couple of times that fit your schedule and I’ll adjust.

Best regards,
Seán


## Approve

In [ ]:
from langgraph.types import Command

response = agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config,  # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='66423253-b4de-4542-9c87-2f6f314d4188'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 595, 'prompt_tokens': 167, 'total_tokens': 762, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DA0gMGSg6vlRkJheDlhEcWPPhtylE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6869-8d2d-7373-a73d-e7d4458b4371-0', tool_calls

## Reject

In [ ]:
response = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán.",
                }
            ]
        }
    ),
    config=config,  # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='66423253-b4de-4542-9c87-2f6f314d4188'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 595, 'prompt_tokens': 167, 'total_tokens': 762, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DA0gMGSg6vlRkJheDlhEcWPPhtylE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6869-8d2d-7373-a73d-e7d4458b4371-0', tool_calls

In [ ]:
print(response["__interrupt__"][0].value["action_requests"][0]["args"]["body"])

KeyError: '__interrupt__'

## Edit

In [ ]:
response = agent.invoke(
    Command(
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    },
                }
            ]
        }
    ),
    config=config,  # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response immediately. Send the reply now in the same thread.', additional_kwargs={}, response_metadata={}, id='66423253-b4de-4542-9c87-2f6f314d4188'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 595, 'prompt_tokens': 167, 'total_tokens': 762, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 576, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-DA0gMGSg6vlRkJheDlhEcWPPhtylE', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6869-8d2d-7373-a73d-e7d4458b4371-0', tool_calls